# GPU memory usage for deep learning image processing

### 目的
DLによる画像処理について、モデルの学習と実行に必要なGPUメモリー容量の理論値と実測値の差を検証する。

### 方法
以下の４つの項目について、メモリーの使用量の理論値と実測値を比較する。
   
1. 変数
2. Forward Neural Network
3. Convolution Neural Network
4. Vision Transformer
5. UNETR

メモリーの使用量の実測値は、pytorchもしくはtensorflowの関数と、pynvml（nvidia  management library）の関数で測定する。
pytorchやtensorflowの関数ではプログラム中で確保された容量が測定される。一方、nvmlではプログラム実行に必要なpythonやcudaなどのオーバーヘッドを含んだ容量が測定される。
変数とUNETRについては２つの実装方法（pytorch, tensorflow）で実測値を測定する。どちらの実装方法でも実測値に大きな差はないことが期待される。

### 結果
|  | 理論値[MiB] | 実測値[MiB] | 実測値（pynvml）[MiB] |
|---|---|---|---|
| 変数 | 1024 | 1024 | 1832 (pytorch)<br> 2153 (tensorflow) |

#### 検証環境
- python 3.10
- tensorflow 2.13.0
- torch 2.0.1
- NVIDIA driver 530.30.02
- CUDA toolkit 11.8
- cuDNN 8.9

#### 参考文献
1. [A comprehensive guide to memory usage in PyTorch](https://medium.com/deep-learning-for-protein-design/a-comprehensive-guide-to-memory-usage-in-pytorch-b9b7c78031d3)
2. [Estimating GPU Memory Consumption of Deep Learning Models](https://2020.esec-fse.org/details/esecfse-2020-industry-papers/5/Estimating-GPU-Memory-Consumption-of-Deep-Learning-Models), [video](https://dl.acm.org/doi/10.1145/3368089.3417050)

## 変数
float32型（4byte）で0.25GiB次元の変数は、理論値で1GiBのサイズである。
この変数を定義または削除したときのメモリーの使用量を確認して、実測値を求める。

まず、Pytorchでの実測値を求める。

In [3]:
import pynvml
import torch


def print_memory_torch(prefix: str) -> None:
# Print memory usage

    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)    
    memory_al = torch.cuda.memory_allocated()
    memory_res = torch.cuda.memory_reserved()
    memory_maxal = torch.cuda.max_memory_allocated()

    print(f"{prefix}: allocated = {memory_al/1024**2:.3f} MiB, "
        f"reserved = {memory_res/1024**2:.3f}MiB, "
        f"max allocated = {memory_maxal/1024**2:.3f} MiB, "
        f"used = {info.used/1024**2:.3f} MiB")


# Define a variable with 1GiB.
dim = 1024**3//4
var_cpu = torch.zeros(dim, dtype=torch.float32)
print(f"var_cpu dtype: {var_cpu.dtype}, dim: {dim/1024**2}MiB")

torch.cuda.reset_peak_memory_stats()
print_memory_torch("Initial")

# Copy the variable to gpu.
var_gpu = var_cpu.to("cuda")
print_memory_torch("Define")

# Delete the variable from gpu.
del var_gpu
print_memory_torch("Delete")

# Release cached memory.
torch.cuda.empty_cache()
print_memory_torch("Release")


var_cpu dtype: torch.float32, dim: 256.0MiB
Initial: allocated = 0.000 MiB, reserved = 0.000MiB, max allocated = 0.000 MiB, used = 3727.750 MiB
Define: allocated = 1024.000 MiB, reserved = 1024.000MiB, max allocated = 1024.000 MiB, used = 4755.875 MiB
Delete: allocated = 0.000 MiB, reserved = 1024.000MiB, max allocated = 1024.000 MiB, used = 4755.875 MiB
Release: allocated = 0.000 MiB, reserved = 0.000MiB, max allocated = 1024.000 MiB, used = 3731.875 MiB


変数を定義した後に1024MiB=1GiBのVRAMが確保されたので、実測値は理論値と一致した。
delで変数を削除したあともメモリーはreservedとして確保されており、torch.cuda.empty_cache()によって完全にメモリーが解放された。

nvmlのusedの結果から、実際には変数のサイズ以上のメモリーが確保され、変数の削除後にもオーバーヘッド分が残っていた。

次にTensorFlowでの実測値を求める。
Tensorflowはデフォルトでプログラム開始時にメモリを最大限確保する。必要なメモリだけを確保するために、memory growthを有効にする。

In [2]:
import tensorflow as tf
import numpy as np


def print_memory_tf(prefix: str) -> None:
# Print memory usage

    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)  
    memory_info = tf.config.experimental.get_memory_info("GPU:0")

    print(f"{prefix}: current = {memory_info['current']/1024**2:.1f} MiB, "
        f"peak = {memory_info['peak']/1024**2:.1f}MiB, "
        f"used = {info.used/1024**2:.3f} MiB")


# Enabled memory growth.
devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)

# Define a variable with 1GiB.
dim = 1024**3//4
var_cpu = np.zeros(dim, dtype=np.float32)
print(f"var_cpu dtype: {var_cpu.dtype}, dim: {dim/1024**2}MiB")

tf.config.experimental.reset_memory_stats("GPU:0")
print_memory_tf("Initail")

# Copy the variable to gpu.
with tf.device("GPU"):
    var_gpu = tf.constant(var_cpu)
print_memory_tf("Define")

# Delete the variable from gpu.
del var_gpu
print_memory_tf("Delete")

2023-08-03 22:41:43.433478: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 22:41:43.455502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 22:41:43.901319: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-03 22:41:44.373948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

var_cpu dtype: float32, dim: 256.0MiB
Initail: current = 0.0 MiB, peak = 0.0MiB, used = 1676.500 MiB
Define: current = 1024.0 MiB, peak = 1024.0MiB, used = 3724.500 MiB
Delete: current = 0.0 MiB, peak = 1024.0MiB, used = 3724.500 MiB


変数を定義した後に1GiBのVRAMが確保されたので、実測値は理論値と一致した。
オーバーヘッドも含めると、実際には変数のサイズ以上のメモリが確保され、変数を削除したあとも解放されなかった。

## Feedforward Neural Network
画像分類をするFNNのメモリー使用量の理論値と実測値を比較する。

データセットは、10クラス画像分類のためのCIFAR10を用いる。
入力画像は32x32x3次元である。データセットの60000枚のうち、40000枚を学習に、残りを10000枚ずつ検証とテストに割り当てる。
バッチサイズは32とし、学習の1エポックで40000/32=1250回パラメーターが更新される。

FNNは２つの隠れ層を持つ構造とする。
隠れ層は512次元とし、バイアスを持たせず、バッチ正規化を行う。
出力層は分類するクラスと同じ数の10次元とし、バイアスを持たせる。
活性化関数にはReLUを用いる。
学習の最適化にはSGDを用いる。
数値の精度はPytorchのデフォルトであるfloat32とする。

このFNNに必要なメモリーの理論値を求める。
参考文献1によれば、学習と推論に必要なメモリーは各々以下のようになる。

$$
\begin{align}
\text{usage for training}   &= \text{model} + \text{forward} + \text{gradients} + \text{gradient moments} \\
                            &= m + f * \text{batch size} * b + m * d + m * o \\
\text{usage for inference}  &= m \\
\end{align}
$$

ここで、  
$m$: モデルの学習のパラメーターのメモリー使用量  
$f$: 入力層を含む各層の出力のメモリー使用量 
$b$: 各層の出力のみ半精度にするmixed precisionを利用する場合は0.5、そうでない場合は1  
$d$: 複数のGPUで実行する場合は2、そうでない場合は1  
$o$: 最適化で使うモーメントの数（SGD: 0, Adagrad, RMSprop: 1, Adam: 2）

FNNの場合は、  
$m = 32*32*3*512 + 512*2 + 512*512 + 512*2 + 512*10+10$ （隠れ層1＋バッチ正規化1＋隠れ層2＋バッチ正規化2＋出力層）  
$f = 512*32*2$ （次元数xバッチサイズx活性化関数の数）  
